# 🚀 Fabric Spark Advisor - Deployment Guide

## Choose Your Deployment Strategy

| Option | Best For | Setup Time | Cost |
|--------|----------|------------|------|
| **1. ngrok Tunnel** | Dev/Testing | 2 min | Free |
| **2. Azure Container Apps** | Production | 15 min | $10-30/mo |
| **3. In-Notebook** | Demos/Offline | 1 min | Free |

---

## ✅ Install Package (Required for All Options)

In [ ]:
# Install the wheel package
%pip install fabric-spark-advisor

# Or install from local wheel file
# %pip install /path/to/fabric_spark_advisor-0.1.0-py3-none-any.whl

---

# 🔧 Option 1: ngrok Tunnel (Dev/Testing)

**Use Case:** Quick development, testing, demos  
**Prerequisites:** Local server running + ngrok installed

### On Your Local Machine:

```powershell
# Terminal 1: Start MCP server
python spark_mcp_server.py

# Terminal 2: Create tunnel
ngrok http 8000
# Copy the Forwarding URL: https://abc123.ngrok-free.app
```

### In This Fabric Notebook:

In [ ]:
from fabric_spark_advisor import SparkAdvisor

# 🔴 REPLACE WITH YOUR NGROK URL
NGROK_URL = "https://abc123def456.ngrok-free.app"

advisor = SparkAdvisor(mcp_server_url=NGROK_URL)
advisor.launch()

### ⚠️ ngrok Limitations

- URL changes every restart (free tier)
- 40 connections/minute limit
- Must keep local server running

**Solution:** Upgrade to ngrok Pro ($8/mo) for static domains

---

# ☁️ Option 2: Azure Container Apps (Production)

**Use Case:** Team usage, always available, scalable  
**Prerequisites:** Azure subscription, deployed container

See full deployment guide: `deployment/azure-container-apps-deploy.md`

### Quick Deploy (Azure CLI):

```bash
# See azure-container-apps-deploy.md for full script
az containerapp create \
  --name spark-advisor-mcp \
  --resource-group spark-advisor-rg \
  --environment spark-advisor-env \
  --image youracr.azurecr.io/spark-advisor-mcp:latest \
  --target-port 8000 \
  --ingress external
```

### In This Fabric Notebook:

In [ ]:
from fabric_spark_advisor import SparkAdvisor

# 🔴 REPLACE WITH YOUR AZURE CONTAINER APP URL
AZURE_URL = "https://spark-advisor-mcp.eastus.azurecontainerapps.io"

advisor = SparkAdvisor(mcp_server_url=AZURE_URL)
advisor.launch()

### ✅ Production Benefits

- Static URL (never changes)
- Auto-scaling (handles load)
- High availability (99.95% SLA)
- Team access (everyone uses same URL)
- Monitoring/logging built-in

---

# 📓 Option 3: In-Notebook Server (Self-Contained)

**Use Case:** Demos, offline work, no external dependencies  
**Prerequisites:** None - runs entirely in this notebook

## Step 1: Configure Credentials

Use Fabric Key Vault for secure credential management:

In [ ]:
import os
from notebookutils import mssparkutils

# 🔴 REPLACE 'your-keyvault' with your Fabric Key Vault name
KEYVAULT_NAME = "your-keyvault"

# Kusto credentials
os.environ["KUSTO_CLUSTER_URL"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "kusto-cluster-url")
os.environ["KUSTO_DATABASE"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "kusto-database")
os.environ["KUSTO_CLIENT_ID"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "kusto-client-id")
os.environ["KUSTO_CLIENT_SECRET"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "kusto-client-secret")
os.environ["KUSTO_TENANT_ID"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "kusto-tenant-id")

# Azure AI Search
os.environ["AZURE_SEARCH_ENDPOINT"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "search-endpoint")
os.environ["AZURE_SEARCH_KEY"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "search-key")
os.environ["AZURE_SEARCH_INDEX"] = "spark-docs"

# Azure OpenAI
os.environ["AZURE_OPENAI_ENDPOINT"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "openai-endpoint")
os.environ["AZURE_OPENAI_API_KEY"] = mssparkutils.credentials.getSecret(KEYVAULT_NAME, "openai-key")
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-4o"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"

print("✅ Credentials configured from Key Vault")

## Step 2: Launch In-Notebook Advisor

In [ ]:
from fabric_spark_advisor import LocalSparkAdvisor

# Initialize advisor (no server needed!)
advisor = LocalSparkAdvisor(session_id="my-fabric-notebook")

# Launch Gradio UI inline
advisor.launch_ui()

## Step 3: Use Programmatic API

In [ ]:
# Analyze specific application
result = await advisor.analyze_application("application_1771446566369_0001")

print(f"Health: {result['overall_health']}")
print(f"Summary: {result['summary']}")
print(f"\nTop 3 Recommendations:")
for i, rec in enumerate(result['validated_recommendations'][:3], 1):
    print(f"{i}. [{rec['source'].upper()}] {rec['recommendation'][:100]}...")

In [ ]:
# Analyze skew
skew_result = await advisor.analyze_skew("application_1771446566369_0001")
print(skew_result)

In [ ]:
# Check scaling impact
scaling_result = await advisor.analyze_scaling("application_1771446566369_0001")
print(scaling_result)

In [ ]:
# Free-form chat
response = await advisor.chat("Show me the top 5 slowest applications")
print(response)

---

# 🎯 Comparison Summary

## When to Use Each Option:

### Use **ngrok** if:
- You're developing/testing locally
- Need quick feedback cycles
- Only you will use it
- Don't mind restarting tunnel

### Use **Azure Container Apps** if:
- Multiple team members need access
- Want production-grade reliability
- Need monitoring/logging
- Can spend $10-30/month

### Use **In-Notebook** if:
- Demoing to stakeholders
- No external dependencies allowed
- Offline work environment
- Simple one-time analysis

---

## 🔐 Security Best Practices

1. **Never hardcode credentials** in notebooks
2. Use **Fabric Key Vault** for secrets
3. Enable **Managed Identity** when possible
4. Use **HTTPS only** (all options support it)
5. Restrict **network access** (Azure NSG/Firewall)

---

## 📚 Additional Resources

- **Full Deployment Guide:** `/deployment/azure-container-apps-deploy.md`
- **ngrok Setup:** `/deployment/ngrok-tunnel-setup.md`
- **In-Notebook Guide:** `/deployment/in-notebook-server.md`
- **API Reference:** `/fabric_spark_advisor/README.md`

---

## 💡 Next Steps

1. Choose your deployment option above
2. Follow the setup steps
3. Start analyzing Spark applications!
4. Share feedback: https://github.com/your-repo/issues